### 1. Import packages

In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: rant3. Use `wandb login --relogin` to force relogin


True

In [2]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")
# use cpu run
import gym


### 2. Helper functions

In [3]:
def dict2array(state):
    new_state = []
    for key  in state.keys():
        if key != 'sw':
            new_state.append(state[key])
        else:
            new_state += list(state['sw'])        
    state = np.asarray(new_state)
    return state

### 3. Initialize the environment

In [4]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = 35
#ram_dimnetion = 9 if partial
nb_actions = 25
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)
# observation has 27 elements, 9 values in soil water
# state size = 27+8 dimension
# how to defind nb_action? why is 200?

Observation: {'cleach': 0.0, 'cnox': 0.0, 'cumsumfert': 0.0, 'dap': 0, 'dtt': 0.0, 'es': 0.0, 'grnwt': 0.0, 'istage': 7, 'nstres': 0.0, 'pcngrn': 0.0, 'pltpop': 7.199999809265137, 'rain': 0.0, 'rtdep': 0.0, 'runoff': 0.0, 'srad': 13.300000190734863, 'sw': array([0.086     , 0.086     , 0.086     , 0.086     , 0.086     ,
       0.076     , 0.076     , 0.13      , 0.25799999]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 22.200000762939453, 'tmin': 3.299999952316284, 'tnoxd': 0.0, 'topwt': 0.0, 'trnu': 0.0, 'vstage': 0.0, 'wtdep': 0.0, 'wtnup': 0.0, 'xlai': 0.0}
27 9

Ram information received from DASSAT will has 35 dimensions.
There are 25 possible actions at each step.
Discrete? False


### 4. Define the network

In [5]:
class QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=256, fc2_units=256,fc3_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, fc3_units)
        self.fc4 = nn.Linear(fc3_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        y = F.relu(self.fc2(x))
        z = F.relu(self.fc3(y))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return self.fc4(z)

In [6]:
trained_model = QNetwork(35, 25)
trained_model.load_state_dict(torch.load('/home/rant3/focal/IAAI/Trained_Policies/FL_Economic_Full/1.pth'))

<All keys matched successfully>

### 5. Define the Replay Buffer

In [7]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            how many samples stored in buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        # store [s,a,r,s',done] for each sample in buffer
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
#         action = dict2array(action)
#         print('transfored actions:',action)
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
#         for e in experiences:
# #             print(e)
# #             print(e.state.shape, e.next_state.shape)
#             break
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        # vstack is add rows together as a single matrix
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        #get all the states, actions, rewards, next_states, dones for all elements in this sample batch, each as a single matrix
        #device here is cpu?
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 6. Define the Agent

In [8]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 640         # minibatch size
GAMMA = 0.99             # discount factor
TAU = 60                # for update of target network parameters
LR = 1e-5               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

In [11]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        # add current (s,r,a,s') and sample a batch and learn if update_every
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step += 1
        if self.t_step%UPDATE_EVERY == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps):
        """Returns actions for given state as per current policy.
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()
#         Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        # return the action index with the largest value
        else:
            return random.choice(np.arange(self.action_size))
            #return one action randomly with possibility eps

#         Epsilon-greedy action selection
#         if random.random() > eps:
#             return np.argmax(action_values.cpu().data.numpy())
#         else:
#             return random.choice(np.arange(self.action_size))
# #         return action_values.cpu().data.numpy()


    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences
        
        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # detach for diemnsion correctness
        # GET largest value of self.qnetwork_target(next_states)
#         print(Q_targets_next)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # 0 if done

        # Get expected Q values from local model
#         print('out',self.qnetwork_local(states).shape)
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        #Why need to gather? actions are indexs?

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        # after this, the parameter of local network will change based on gradient descent
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)
        # stabilize traning to keep grad between (-1,1)
        self.optimizer.step()
        #固定用法, change learning rate

        #Update target network weights every TAU learning steps (so every TAU*UPDATE_EVERY t_step)
        if self.t_step%(TAU*UPDATE_EVERY)==0:
            self.update_target_net(self.qnetwork_local, self.qnetwork_target)                     

    def update_target_net(self, local_model, target_model):
        """Soft update model parameters.
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
        """
        target_model.load_state_dict(local_model.state_dict())
    def save(self,name):
        torch.save(self.qnetwork_local.state_dict(),'/home/rant3/focal/model'+name+'.pth')
        print('saved')

In [12]:
def get_reward(state, n_action, w_action, next_state, done, k1, k2, k3, k4, n_amount, w_amount):
    #if done, return Yield (topwt) - k*cost
    penalty1 = 0
    penalty2 = 0
    #k1=10**(-3)*(i_episode*2)
    # base cost is current input action
    if n_amount > 250:
        if n_action!=0:
            penalty1 = (n_amount-250)
    if w_amount > 600:
        if w_action!= 0:
            penalty2=(w_amount-600)
    if done:
        reward = state[29] - k1*n_action - k2*w_action- k3*penalty1-k4*penalty2
        return reward
    #if done, return Yield (topwt) - k*cost
    else:
        reward = -k1*(n_action + state[25]) - k2*w_action- k3*penalty1-k4*penalty2
        return reward
    # otherwise, return -k*(action+leaching)

In [13]:
def get_reward(state, n_action, w_action, next_state, done, k1, k2, k3, k4):
    if done:
        reward = k1*state[6] - k2*n_action - k3*w_action- k4*state[25]
        return reward
    #if done, return Yield (topwt) - k*cost
    else:
        reward = -k2*n_action - k3*w_action - k4*state[25] 
        return reward
    # otherwise, return -k*(action+leaching)

In [14]:
def get_economic_reward(state, n_action, w_action, next_state, done, k1, k2, k3):
    if done:
        reward = k1*state[6] - k2*n_action - k3*w_action
        return reward
    #if done, return Yield (topwt) - k*cost
    else:
        reward = -k2*n_action - k3*w_action 
        return reward

In [15]:
def get_effi_reward(state, n_action, w_action, next_state, done, k1, k2, k3):
    if state[2] != 0:
        reward = state[6]/state[2]
        return reward
    else:
        reward = -30
        return reward

### DQN for nitrogen management

In [14]:
agent = Agent(state_size=ram_dimensions, action_size=nb_actions)

In [15]:
def dqn(n_episodes=2000, max_t=500, eps_start=1.0, eps_end=0, eps_decay=0.994, solved=8000, optimize = True,exp=0):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    start = time.time()                # Start time
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    list_eps = []
    eps = eps_start                    # initialize epsilon
    n_amount = 0
    w_amount = 0
    n_amount_list=[]
    n_amount_window = deque(maxlen=100)
    w_amount_list=[]
    w_amount_window = deque(maxlen=100)
    leaching_sum_list = []
    leaching_sum_window = deque(maxlen=100)
    yield_list=[]
    yield_window=deque(maxlen=100)
    y=0
    k1=0.2
    k2=1
    k3=1
    k4=10
    wandb.init(
      # Set the project where this run will be logged
      project="FL_RF3", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{exp}", config={'k1':k1,'k2':k2,'k3':k3,'k4':k4,'Decay':eps_decay,'Gamma':GAMMA} )
      # Track hyperparameters and run metadata)
    for i_episode in range(1, n_episodes+1):
        # Reset env and score at the beginning of episode
        state = env.reset() # reset the environment and get current state
        state = dict2array(state)
        score = 0 # initialize the score
        n_amount = 0
        w_amount = 0
        leaching_sum = 0
        for t in range(max_t):
            action1 = agent.act(state, eps) if optimize else 12
            action = {
                    'anfer': (action1%5)*40,  # if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha
                    'amir': int(action1/5)*6,  # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
            next_state, reward, done, _ = env.step(action)      # send the action to the environment
            #reward = reward-
            # add changes to reward to penalize multiple actions
            # this step is different from agent step
            if done:
                y=state[6]
                next_state = state
                reward = get_reward(state, action['anfer'], action['amir'], next_state, done, k1, k2, k3, k4)
                agent.step(state, action1, reward, next_state, done)
                score += reward
                n_amount_list.append(n_amount)
                w_amount_list.append(w_amount)
                leaching_sum=env.observation['cleach']
                leaching_sum_list.append(leaching_sum)
                #print('Episode is ', i_episode, 'Yield is',state[29])
                n_amount_window.append(n_amount)
                w_amount_window.append(w_amount)
                leaching_sum_window.append(leaching_sum)
                yield_list.append(y)
                yield_window.append(y)
                break
            n_amount += action['anfer']
            w_amount += action['amir']
            next_state = dict2array(next_state)
            reward = get_reward(state, action['anfer'], action['amir'], next_state, done, k1, k2, k3, k4)
            agent.step(state, action1, reward, next_state, done)
            state = next_state
            score += reward
            #append
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        if score > 1460:
            agent.save(str(i_episode))
        
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        list_eps.append(eps)
        #if eps<0.01:
        #    if score > 1900:
        #        agent.save(str(i_episode))
        #        print('save')
        #if i_episode>600:
        #    if i_episode%40 ==0:
        #        agent.save(str(i_episode))
        #if i_episode > 1400:
        #    eps = 0
        if i_episode % 100 == 0:
            print('\rEpisode {}/{} \tAverage Score: {:.2f}'.format(i_episode, n_episodes, np.mean(scores_window)))
            print('Epsilon: {}'.format(eps))
            print('Average nitrogen amount is', np.mean(n_amount_window))
            print('Average irrigation amount is', np.mean(w_amount_window))
            print('Average leaching sum is', np.mean(leaching_sum_window))
            print('Average yield is', np.mean(yield_window))
            
        if np.mean(scores_window)>solved:
            print('Game Solved after {} episodes'.format(i_episode))
            break
        wandb.log({"reward": score, "N_amount": n_amount,"W_amount":w_amount,"leaching":leaching_sum,'yield':y})
    time_elapsed = time.time() - start
    print("Time Elapse: {:.2f} seconds".format(time_elapsed))
    
    return scores, list_eps, n_amount_list, w_amount_list, leaching_sum_list

In [16]:
scores, list_eps, n_amount_list, w_amount_list, leaching_sum_list = dqn(n_episodes=8000,exp=1)

Episode 100/8000 	Average Score: -85558.18
Epsilon: 0.5478207015323072
Average nitrogen amount is 10027.2
Average irrigation amount is 1774.8
Average leaching sum is 7591.45623046875
Average yield is 11297.92865578522
Episode 200/8000 	Average Score: -41579.47
Epsilon: 0.3001075210273491
Average nitrogen amount is 5273.2
Average irrigation amount is 1077.42
Average leaching sum is 3744.3024072265625
Average yield is 11293.358090720752
Episode 300/8000 	Average Score: -20295.64
Epsilon: 0.164405112704324
Average nitrogen amount is 2950.0
Average irrigation amount is 692.04
Average leaching sum is 1888.2570483398438
Average yield is 11266.430161746599
Episode 400/8000 	Average Score: -8623.09
Epsilon: 0.09006452417718083
Average nitrogen amount is 1497.2
Average irrigation amount is 514.92
Average leaching sum is 879.0728882217408
Average yield is 11003.617907966554
Episode 500/8000 	Average Score: -4177.95
Epsilon: 0.04933921081791662
Average nitrogen amount is 892.8
Average irrigation 

Episode 3900/8000 	Average Score: 606.05
Epsilon: 6.410606942315255e-11
Average nitrogen amount is 216.8
Average irrigation amount is 299.58
Average leaching sum is 91.7388126373291
Average yield is 10201.705360850608
Episode 4000/8000 	Average Score: 649.44
Epsilon: 3.5118631923870196e-11
Average nitrogen amount is 202.4
Average irrigation amount is 299.46
Average leaching sum is 88.62015296936035
Average yield is 10191.313184612236
Episode 4100/8000 	Average Score: 672.38
Epsilon: 1.9238713577389448e-11
Average nitrogen amount is 202.4
Average irrigation amount is 299.94
Average leaching sum is 87.41113998413086
Average yield is 10245.056619467818
Episode 4200/8000 	Average Score: 698.41
Epsilon: 1.0539365568544608e-11
Average nitrogen amount is 208.8
Average irrigation amount is 305.16
Average leaching sum is 85.83704643249511
Average yield is 10356.725050347502
Episode 4300/8000 	Average Score: 680.52
Epsilon: 5.77368263946555e-12
Average nitrogen amount is 209.2
Average irrigation

Episode 7700/8000 	Average Score: 875.74
Epsilon: 7.501702884522507e-21
Average nitrogen amount is 250.4
Average irrigation amount is 278.58
Average leaching sum is 76.17584266662598
Average yield is 10841.61141933834
Episode 7800/8000 	Average Score: 874.54
Epsilon: 4.109588136886052e-21
Average nitrogen amount is 252.4
Average irrigation amount is 279.0
Average leaching sum is 76.28460853576661
Average yield is 10854.829586712787
Episode 7900/8000 	Average Score: 865.84
Epsilon: 2.2513174561577645e-21
Average nitrogen amount is 230.4
Average irrigation amount is 280.68
Average leaching sum is 76.64792762756348
Average yield is 10719.89570600997
Episode 8000/8000 	Average Score: 865.56
Epsilon: 1.2333183082042758e-21
Average nitrogen amount is 238.8
Average irrigation amount is 279.18
Average leaching sum is 76.37161506652832
Average yield is 10737.477247292267
Time Elapse: 8244.17 seconds


wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.
wandb: Network error (ReadTimeout), entering retry loop.


In [14]:
import pandas as pd
#print(len(scores))
#action_list=action_list/100
a = pd.DataFrame(scores)
a.to_csv('4000 episode_large buffer.csv', index=False, header=False)
#b = pd.DataFrame(action_list)
#b.to_csv('Action with constant k2=0.5.csv', index=False, header=False)

### Given a constant input: reward = 19847

In [15]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(scores[-3000:])
plt.title("3000")
plt.xlabel("Traing Steps")
plt.ylabel("Cumulative Rewards")
plt.savefig("3000.png")

In [18]:
plt.plot(scores)
plt.yscale('symlog')
plt.title("Nitrogen Management")
plt.xlabel("Traing Steps")
plt.ylabel("Cumulative Rewards")
plt.savefig("Cumulative reward (log scale) linear penalty (partial observability).png")

In [18]:
plt.figure()
print(action_list)
plt.plot(action_list)
plt.title("Naction of last episode")
plt.xlabel("Days")
plt.ylabel("Nitrogen input")
plt.savefig("Action with constant k2=0.5.png")

[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.1  0.9  0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.04
 0.   0.   0.   0.04 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.  ]


# 

In [20]:
plt.figure()
print(np.mean(scores[-90:]))
plt.plot(scores[-90:])
plt.savefig("Reward with constant k2=0.5.png")

479.7642289510514
